In [1]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

%matplotlib inline
import pandas as pd
pd.options.display.max_columns = 100
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np

from sklearn import base

pd.options.display.max_rows = 100

In [2]:
pwd

u'C:\\Users\\cnyi\\Box Sync\\Kaggle\\titanic'

In [3]:
data = pd.read_csv('data/train.csv')

In [4]:
data.describe() # Age is missing

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
from sklearn import base

class Createdummy(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self

    
    def transform(self, X):
        import pandas as pd
        X=pd.DataFrame(X)
        new_data=pd.get_dummies(X[self.column_name])
        new_data=X.join(new_data)
        new_data.drop(self.column_name,1,inplace=True)
        return new_data

In [7]:
from sklearn import base

class CreateTitle(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self

    
    def transform(self,X):    
        # we extract the title from each name
        X['Title'] = X[self.column_name].map(lambda name:name.split(',')[1].split('.')[0].strip())

        # a map of more aggregated titles
        Title_Dictionary = {
                            "Capt":       "Officer",
                            "Col":        "Officer",
                            "Major":      "Officer",
                            "Jonkheer":   "Royalty",
                            "Don":        "Royalty",
                            "Sir" :       "Royalty",
                            "Dr":         "Officer",
                            "Rev":        "Officer",
                            "the Countess":"Royalty",
                            "Dona":       "Royalty",
                            "Mme":        "Mrs",
                            "Mlle":       "Miss",
                            "Ms":         "Mrs",
                            "Mr" :        "Mr",
                            "Mrs" :       "Mrs",
                            "Miss" :      "Miss",
                            "Master" :    "Master",
                            "Lady" :      "Royalty"

                            }

        # we map each title
        X['Title'] = X.Title.map(Title_Dictionary)
        return X

In [8]:
from sklearn import base

class dropcolumn(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self

    
    def transform(self,X):
        data_new=X.drop(self.column_name,1)
        return data_new
        

In [78]:
from sklearn import base

class PrepareData(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        try:
            X=X[X.columns.drop(self.column_name)]
        #return X
            return X.values
        except:
            return X.values
        


In [10]:
from sklearn import base

class fill_nan(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,column_name):
        self.column_name=column_name
        
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.column_name].fillna(X[self.column_name].median(), inplace=True)
        return X
   

In [11]:
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

titanic_Ridge=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('Ridge',Ridge())
    ])

Ridge_parameters = {
    'Ridge__alpha':  10**np.linspace(15,-5,400)*0.5  
}

In [14]:
from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_Ridge, Ridge_parameters, verbose=1)

print("Performing grid search...")
print("titanic_Ridge:", [name for name, _ in titanic_Ridge.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(Ridge_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
('titanic_Ridge:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'Ridge'])
parameters:
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
()
Best score: 0.404
Best parameters set:
	Ridge__alpha: 6.5201595759206707


[Parallel(n_jobs=1)]: Done 1200 out of 1200 | elapsed:  9.3min finished


In [15]:
titanic_Ridge_best=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('Ridge',Ridge(alpha=6.5201595759206707))
    ])


In [18]:
titanic_Ridge_best.fit(data,data['Survived'].values)

Pipeline(steps=[('fill_nan', fill_nan(column_name='Age')), ('title', CreateTitle(column_name='Name')), ('drop', dropcolumn(column_name=['Name', 'Ticket', 'Cabin'])), ('dummy', Createdummy(column_name=['Title', 'Sex', 'Embarked'])), ('PrepareData', PrepareData(column_name='Survived')), ('Ridge', Ridge(alpha=6.52015957592, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [28]:
titanic_Ridge_best.steps[5][1].feature_importances_

AttributeError: 'Ridge' object has no attribute 'feature_importances_'

In [30]:
from sklearn.ensemble import RandomForestRegressor
titanic_rf=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('rf',RandomForestRegressor())
    ])

rf_parameters = {
    'rf__max_depth':  [2,15]
}

In [31]:
from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(titanic_rf, rf_parameters, verbose=1)

print("Performing grid search...")
print("titanic_rf:", [name for name, _ in titanic_rf.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(rf_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
('titanic_rf:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'rf'])
parameters:
Fitting 3 folds for each of 2 candidates, totalling 6 fits
()
Best score: 0.382
Best parameters set:
	rf__max_depth: 2


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    3.3s finished


In [33]:
titanic_rf_best=\
Pipeline([
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))
    ,('rf',RandomForestRegressor(max_depth= 2))
    ])


In [34]:
titanic_rf_best.fit(data,data['Survived'].values)

Pipeline(steps=[('fill_nan', fill_nan(column_name='Age')), ('title', CreateTitle(column_name='Name')), ('drop', dropcolumn(column_name=['Name', 'Ticket', 'Cabin'])), ('dummy', Createdummy(column_name=['Title', 'Sex', 'Embarked'])), ('PrepareData', PrepareData(column_name='Survived')), ('rf', RandomForestRegr...timators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [40]:
titanic_rf_best.steps[5][1].feature_importances_

array([ 0.        ,  0.19609464,  0.00989171,  0.01575089,  0.        ,
        0.01986139,  0.00723564,  0.        ,  0.45796736,  0.        ,
        0.        ,  0.        ,  0.22591025,  0.06728813,  0.        ,
        0.        ,  0.        ])

In [41]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
#         What needs to be done here?
#         self.estimator = CityEstimator()
        self.estimator = estimator
    
    def fit(self, X, y):
        
        # Fit the stored estimator.
        # Question: what should be returned?
        self.estimator.fit(X, y)
        return(self)
    
    def transform(self, X):
        # Use predict on the stored estimator as a "transformation".
        # Be sure to return a 2-D array.
        new_list = []
        new_array = self.estimator.predict(X)
        for i in new_array:
            ii = [i]
            new_list.append(ii)
        return(new_list)

In [91]:
from sklearn.pipeline import FeatureUnion

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

pipeline_full = Pipeline([
    # Extract the subject & body
    ('fill_nan',fill_nan(['Age','Fare']))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))

    # Use FeatureUnion to combine the features from subject and body
    ,('union', FeatureUnion(
        transformer_list=[
               ('Ridge_best', EstimatorTransformer(Ridge(alpha=6.5201595759206707)))

              ,('rf_best',EstimatorTransformer(RandomForestRegressor(max_depth=2)))
        ]
    ,transformer_weights={
            'Ridge_best': 1,
            'rf_best': 1,
    }
    ))
   
    # Use a GBT Regressor on the combined features
    ,('GBT', GradientBoostingRegressor()),
    ])



GBT_parameters = {
    'GBT__max_depth':  [2,15]
}

In [66]:
from sklearn.model_selection import GridSearchCV    
grid_search = GridSearchCV(pipeline_full, GBT_parameters, verbose=1)

print("Performing grid search...")
print("pipeline_full:", [name for name, _ in pipeline_full.steps])
print("parameters:")
grid_search.fit(X=data, y=data['Survived'].values)
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(GBT_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
('pipeline_full:', ['fill_nan', 'title', 'drop', 'dummy', 'PrepareData', 'union', 'GBT'])
parameters:
Fitting 3 folds for each of 2 candidates, totalling 6 fits
()
Best score: 0.418
Best parameters set:
	GBT__max_depth: 2


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    3.5s finished


In [100]:
pipeline_full_best = Pipeline([
    # Extract the subject & body
    ('fill_nan',fill_nan('Age'))
    ,('title', CreateTitle('Name'))
    ,('drop',dropcolumn(['Name','Ticket','Cabin','PassengerId']))
    ,('dummy', Createdummy(['Title','Sex','Embarked']))
    ,('PrepareData',PrepareData('Survived'))

    # Use FeatureUnion to combine the features from subject and body
    ,('union', FeatureUnion(
        transformer_list=[
               ('Ridge_best', EstimatorTransformer(Ridge(alpha=6.5201595759206707)))

              ,('rf_best',EstimatorTransformer(RandomForestRegressor(max_depth=2)))
        ]
    ,transformer_weights={
            'Ridge_best': 1,
            'rf_best': 1,
    }
    ))
   
    # Use a GBT Regressor on the combined features
    ,('GBT', GradientBoostingRegressor(max_depth= 2)),
    ])

In [80]:
pipeline_full_best.fit(X=data, y=data['Survived'].values)

Pipeline(steps=[('fill_nan', fill_nan(column_name='Age')), ('title', CreateTitle(column_name='Name')), ('drop', dropcolumn(column_name=['Name', 'Ticket', 'Cabin'])), ('dummy', Createdummy(column_name=['Title', 'Sex', 'Embarked'])), ('PrepareData', PrepareData(column_name='Survived')), ('union', FeatureUnion(...       presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False))])

In [98]:
test = pd.read_csv('data/test.csv')

test['Fare'].fillna(data['Fare'].median(), inplace=True)
test['Age'].fillna(data['Age'].median(), inplace=True)

In [95]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,29.599282,0.447368,0.392344,35.576535
std,120.810458,0.841838,12.703770,0.896760,0.981429,55.850103
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.471875
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [87]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,0,3,"Moran, Mr. James",male,28.0,0,0,330877,8.4583,NaN,Q,Mr
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Master
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs


In [99]:
pipeline_full_best.predict(test)

array([ 0.11018251,  0.6180593 ,  0.10485072,  0.11018251,  0.49033038,
        0.10485072,  0.6486438 ,  0.12967187,  0.88733108,  0.01454518,
        0.11018251,  0.11773468,  0.98222644,  0.10656491,  0.94780692,
        0.95674772,  0.10116544,  0.10983854,  0.3501973 ,  0.68052671,
        0.26107887,  0.63265853,  0.98222644,  0.34776765,  0.94780692,
        0.01454518,  0.98222644,  0.10983854,  0.34776765,  0.10584294,
        0.11018251,  0.12967187,  0.3501973 ,  0.3984714 ,  0.34776765,
        0.10983854,  0.64106001,  0.49033038,  0.11018251,  0.20525337,
        0.07317773,  0.34776765,  0.19530134,  0.95674772,  0.94780692,
        0.11018251,  0.34776765,  0.10485072,  0.98222644,  0.4816354 ,
        0.34776765,  0.22761816,  0.87576884,  0.92597022,  0.22761816,
        0.19884039,  0.07317773,  0.11018251,  0.09470974,  0.98222644,
        0.10485072,  0.10983854,  0.10485072,  0.68052671,  0.94780692,
        0.95674772,  0.68052671,  0.34776765,  0.34776765,  0.94